In [31]:
# !pip install tensorflow --upgrade


In [32]:
import tensorflow as tf
import numpy as np

In [33]:
!cat '/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/trainDev/NOT_OFF/1000.txt'

@USER @USER ان شاء الله يا غالي حبيبي يا محمد<LF>ودايما متجمعين علي عشق الكيان	NOT_OFF	NOT_HS


In [34]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/trainDev",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
   
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/trainDev",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/My Drive/MasterThesis/Datasets/Offensive2020/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Found 8000 files belonging to 2 classes.
Using 6400 files for training.
Found 8000 files belonging to 2 classes.
Using 1600 files for validation.
Found 2000 files belonging to 2 classes.
Number of batches in raw_train_ds: 200
Number of batches in raw_val_ds: 50
Number of batches in raw_test_ds: 63


In [35]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i].decode('utf-8').strip())
        print(label_batch.numpy()[i])
        print('--------------------------------')

RT @USER: يا رب يا عزيز يا جبار .. انك القادر على كل شيء .. يا رب فرحه اتحاديه تُنسينا كل الهموم والمشاكل الي صارت هالموسم يا رب العباد…	NOT_OFF	NOT_HS
0
--------------------------------
@USER @USER يا جامع يا رقيب يا رب تلقاها	NOT_OFF	NOT_HS
0
--------------------------------
راهنت عليك ولم اخسر ❤️❤️<LF>يا وحش يا جلاد يا كبير<LF>يا مرعب يا قناص يا يصياد<LF>🖤💛🖤💛🖤💛🖤💛<LF>#الاتحاد_النصر URL	NOT_OFF	NOT_HS
0
--------------------------------
RT @USER: كانت أيام يا وطني<LF>زي الأحلام يا وطني 🎶	NOT_OFF	NOT_HS
0
--------------------------------
تمثيل يا رزان تمثيل يا رزان تمثيل يا رزان تمثيل يا رزان تمثيل يا رزان تمثيل يا رزان	NOT_OFF	NOT_HS
0
--------------------------------


In [36]:
# demoji.download_codes()

In [50]:
import string
import re

# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    # standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

In [51]:
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# import string
# import re
# def custom_standardization(input_data):
#     # lowercase = tf.strings.lower(input_data)
#     # stripped_html = tf.strings.regex_replace(lowercase, "<LF>", " ")
#     stripped_html = tf.strings.regex_replace(input_data, "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]", " ")
#     after_remove_punctuation = tf.strings.regex_replace(stripped_html, "[.،,!?؟\\-”“٪ًَ]", " ")
    
#     after_remove_hamza = tf.strings.regex_replace(after_remove_punctuation, "[إأآا]", "ا")
#     after_remove_Altaa = tf.strings.regex_replace(after_remove_hamza, "ة", "ه")
#     after_remove_emoji=tf.strings.regex_replace(after_remove_Altaa, "[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)|\u2069|\u2066]+", " ")
#     after_remove_repeating_char = tf.strings.regex_replace(after_remove_emoji, "r'(.)\1+'", "r'\1\1'")

#     return tf.strings.regex_replace(
#        after_remove_repeating_char, "[%s]" % re.escape(string.punctuation), ""
#     )
   
# max_features = 20000
# embedding_dim = 128
# sequence_length = 500

# vectorize_layer = TextVectorization(
#     # standardize=custom_standardization,
#     max_tokens=max_features,
#     output_mode="int",
#     output_sequence_length=sequence_length,
# )

# text_ds = raw_train_ds.map(lambda x, y: x)
# vectorize_layer.adapt(text_ds)


In [52]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [53]:
# vocab = np.array(vectorize_layer.get_vocabulary())
# vocab[:50]

In [41]:
# vectorize_layer

In [42]:
# encoded_example = vectorize_layer(text_batch)[:50].numpy()
# encoded_example

In [43]:
# for n in range(4):
#   print("Original: ", text_batch[n].numpy().decode('utf-8').strip())
#   print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
#   print()

## Build a model 

In [54]:
# from tensorflow.keras import layers
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(max_features, 64),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1)
# ])

model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [55]:
# print([layer.supports_masking for layer in model.layers])

In [56]:
# # predict on a sample text without padding.

# sample_text = ('وانت يا لص يا متصهين لا تمثل المجتمع الكوردي')
# predictions = model.predict(np.array([sample_text]))
# print(predictions[0])

In [57]:

# # predict on a sample text with padding

# padding = " ال" * 2000
# predictions = model.predict(np.array([sample_text, padding]))
# print(predictions[0])

In [58]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Train the model

In [59]:
history = model.fit(train_ds, 
                    epochs=10,
                    validation_data=val_ds,
                    validation_steps=30
                    )
                  
                  
                    # steps_per_epoch=30
                    

Epoch 1/10


TypeError: ignored

## Evaluate the model on the test set


In [ ]:
test_loss, test_acc = model.evaluate(test_ds)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

## plotlib

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

#epochs_range = range(22)

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# !pip install tensorflow.js